In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# 1. 출력 구조 정의
class StudentInfo(BaseModel):
    name: str = Field(description="학생 이름")
    age: int = Field(description="학생 나이")
    major: str = Field(description="전공")
    hobbies: List[str] = Field(description="취미 리스트")
    goal: str = Field(description="학생의 목표")

# 2. Pydantic 파서 설정
parser = PydanticOutputParser(pydantic_object=StudentInfo)

# 3. 프롬프트 템플릿 정의
template = """
당신은 학생 정보 추출 시스템입니다.
아래 자기소개 문장에서 이름, 나이, 전공, 취미들, 목표를 추출하여 구조화하세요.

자기소개: {intro}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template).partial(
    format_instructions=parser.get_format_instructions()
)

# 4. 모델 설정
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 5. 체인 구성
chain = prompt | model | parser

# 6. 테스트용 자기소개 입력
intro_text = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."

# 7. 실행
output = chain.invoke({"intro": intro_text})

# 8. 결과 출력
print("=== 추출된 학생 정보 ===")
print(f"이름: {output.name}")
print(f"나이: {output.age}")
print(f"전공: {output.major}")
print(f"취미: {', '.join(output.hobbies)}")
print(f"목표: {output.goal}")

=== 추출된 학생 정보 ===
이름: 김민수
나이: 22
전공: 컴퓨터공학
취미: 게임하기, 영화보기, 코딩
목표: 훌륭한 개발자가 되는 것
